In [5]:
%load_ext autoreload
%autoreload 2

# Second cell
from zoom_info_module import zoominfo

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import pandas as pd

In [14]:
test = zoominfo()

In [15]:
firstName = 'Alan'
lastName = 'Raffie'
company = 'ARALCO'

In [16]:
test.search(firstName, lastName, company)

,companyID,companyName,companyAddress,companyRevenue,companyRevenueRange,companyEmployees,companyEmployeeCountRange,companyDomain,website,companyDescription,...,score,mobilePhone,mobilePhoneIsDoNotCall,privacyRegulationFlags,certifications,isEU,creationDate,buyingCommittee,hasOnlinePresence,personHashtags
0,4687530,ARALCO,"{'Street': '8420 Citrus Wynd', 'City': 'Vancou...",$4.8 Million,$1 mil. - $5 mil.,18,10 - 20,www.aralco.com,www.aralco.com,Aralco offers the best power-to-cost ratio on ...,...,1,(604) 644-2816,True,{'privacyRegulationFlag': 'canada'},[{'certificationName': 'President & Chief Exec...,False,1970-01-01,False,True,"['JOB_FUNCTION_0.0_40_TO_49_YRS', '#Flag_likel..."
